In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir("C:\\Users\\faizan\\Documents\\IMLChallenge02")

In [3]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
sample_df = pd.read_csv('./sample_submission.csv')

In [4]:
train_missing_values = train_df.isnull().sum()
train_data_types = train_df.dtypes

test_missing_values = test_df.isnull().sum()
test_data_types = test_df.dtypes

In [5]:
from sklearn.impute import SimpleImputer
X_train = train_df.drop(['price_doc'], axis=1)
y_train = train_df['price_doc']

# Identify numerical and categorical columns
num_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns

# Impute missing values for numerical features with median
imputer_num = SimpleImputer(strategy='mean')
X_train[num_cols] = imputer_num.fit_transform(X_train[num_cols])
test_df[num_cols] = imputer_num.transform(test_df[num_cols])

# Impute missing values for categorical features with mode
imputer_cat = SimpleImputer(strategy='most_frequent')
X_train[cat_cols] = imputer_cat.fit_transform(X_train[cat_cols])
test_df[cat_cols] = imputer_cat.transform(test_df[cat_cols])

# Check if there are any missing values left
X_train.isnull().sum().sum(), test_df.isnull().sum().sum()

(0, 0)

In [6]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Scale the numerical columns using StandardScaler
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

In [7]:
# Perform one-hot encoding for categorical features
X_train_encoded = pd.get_dummies(X_train, columns=cat_cols)
test_df_encoded = pd.get_dummies(test_df, columns=cat_cols)

# Ensure that both training and test data have the same set of columns after one-hot encoding
missing_cols = set(X_train_encoded.columns) - set(test_df_encoded.columns)
for col in missing_cols:
    test_df_encoded[col] = 0

test_df_encoded = test_df_encoded[X_train_encoded.columns]

In [9]:
test_df_encoded.shape

(77789, 2214)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

cat_cols = X_train.select_dtypes(include=['object']).columns

# Apply label encoding to categorical columns in both train and test data
for col in cat_cols:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    test_df[col] = label_encoder.transform(test_df[col])


# Ensure that both training and test data have the same set of columns after one-hot encoding
missing_cols = set(X_train_encoded.columns) - set(test_df_encoded.columns)
for col in missing_cols:
    test_df_encoded[col] = 0

test_df_encoded = test_df_encoded[X_train_encoded.columns]

In [11]:
X_train_encoded.shape

(181507, 2214)

In [10]:
# Model Training
model = LinearRegression()
model.fit(X_train_encoded, y_train)

MemoryError: Unable to allocate 2.99 GiB for an array with shape (2214, 181507) and data type float64

In [ ]:
# Initialize your linear regression model
model = LinearRegression()

# Initialize Recursive Feature Elimination with Cross-Validation (RFECV)
selector = RFECV(estimator=model, step=1, cv=5)

# Fit RFECV on your data
selector = selector.fit(X_train_encoded, y_train)

# Transform your data to include only the selected features
X_train_selected = selector.transform(X_train_encoded)
X_test_selected = selector.transform(X_test_encoded)

# Train your model on the selected features
model.fit(X_train_selected, y_train)

In [ ]:
# Predict using the trained model on the test data
X_test_encoded = pd.get_dummies(test_df, columns=cat_cols)  # One-hot encode the test data
missing_cols_test = set(X_train_encoded.columns) - set(X_test_encoded.columns)

In [ ]:
# Ensure that the test data has the same columns as the training data after one-hot encoding
for col in missing_cols_test:
    X_test_encoded[col] = 0

X_test_encoded = X_test_encoded[X_train_encoded.columns]

In [ ]:
# Scale the numerical columns using StandardScaler
X_test_encoded[num_cols] = scaler.transform(X_test_encoded[num_cols])

# Make predictions
predicted_price = model.predict(X_test_encoded)

In [ ]:
# Create submission DataFrame
submission_df = pd.DataFrame({
    'row ID': test_df['row ID'],
    'price_doc': predicted_price
})

# Save the DataFrame to a CSV file
submission_df.to_csv('prediction_linearRegression.csv', index=False)

In [ ]:
del train_df
del test_df